In [ ]:
using Plots
using Random
using DataFrames
using Statistics

println("a")

function naive_multiplication(A,B)
    println("a")
    C = zeros(Float64, size(A,1), size(B,2))
    for i=1:size(A,1)
        for j=1:size(B,2)
            for k=1:size(A,2)
                C[i,j] = C[i,j] + A[i,k]*B[k,j]
            end
        end
    end
    return C
end

println("a")
function better_multiplication(A,B)
    
    println("a")
    C = zeros(Float64, size(A,1), size(B,2))
    for j=1:size(B,2)
        for k=1:size(A,2)
            for i=1:size(A,1)
                C[i,j] = C[i,j] + A[i,k]*B[k,j]
            end
        end
    end
    return C
end

println("a")
operation = []
length = []
time = []

println(-1)

tmp1 = @elapsed naive_multiplication(rand(2, 2), rand(2, 2))

println(-2)
tmp2 = @elapsed better_multiplication(rand(2, 2), rand(2, 2))

println(-3)

tmp3 = @elapsed rand(2, 2) * rand(2, 2)

println(-4)
for i in 1:10
    println(i)
    for j in 1:10
        A = rand(2^i, 2^i)
        B = rand(2^i, 2^i)
        push!(operation, "naive")
        push!(length, 2^i)
        push!(time, @elapsed naive_multiplication(A, B))
        push!(operation, "better")
        push!(length, 2^i)
        push!(time, @elapsed better_multiplication(A, B))
        push!(operation, "BLAS")
        push!(length, 2^i)
        push!(time, @elapsed A*B)
    end
end

df = DataFrame(operation = operation, length = length, time = time)
print(df)
df1 = df[df.operation .== "naive", :]
df1 = groupby(df1, [:length])
df1 = combine(df1, "time" => mean, "time" => std)
df2 = df[df.operation .== "better", :]
df2 = groupby(df2, [:length])
df2 = combine(df2, "time" => mean, "time" => std)
df3 = df[df.operation .== "BLAS", :]
df3 = groupby(df3, [:length])
df3 = combine(df3, "time" => mean, "time" => std)
plot([df1.length df2.length df3.length], [df1.time_mean df2.time_mean df3.time_mean],
title = "runtime of different interpolation methods",
xlab = "length", ylab = "time [s]",
label = ["naive" "better" "BLAS"],
yerr = [df1.time_std df2.time_std df3.time_std],
colour = [:red :blue :green],
legend = :topleft)
